## STANDARDIZE DATA (ETOP DATASET)


In [7]:
import os
import re
import pandas
from IPython.display import HTML

#### UPDATE LEVEL TO DATASET


In [8]:
# Load dữ liệu từ standard-data.csv
standard_df = pandas.read_csv("./standard-data.csv")
print(f"Standard data shape: {standard_df.shape}")
print(f"Standard data columns: {standard_df.columns.tolist()}")

# Thêm cột level vào standard_df nếu chưa có
if 'level' not in standard_df.columns:
    standard_df['level'] = None

# Định nghĩa mapping từ course name sang level
level_mapping = {
    "etop-beginner": "Beginner",
    "etop-intermediate": "Intermediate", 
    "etop-advanced": "Advanced"
}

# Loop qua tất cả các course levels
for course in data_course:
    print(f"\n=== PROCESSING {course.upper()} ===")
    
    # Load dữ liệu course
    df_course = pandas.read_csv("./data/" + course + ".csv")
    
    # Rename columns
    df_course.rename(
        columns={
            "First name": "firstname",
            "Surname": "lastname", 
            "Username": "username",
        },
        inplace=True,
    )
    
    # Drop unnecessary columns
    df_course.drop(columns=["Groups"], inplace=True)
    
    # Filter by pattern
    pattern = r"[A-Z]{2}[0-9]{2}[A-Z]{1}[0-9]{4}"
    df_course = df_course[df_course["firstname"].str.match(pattern, na=False)]
    df_course = df_course[["username", "firstname", "lastname"]]
    
    print(f"Course {course} data shape after filtering: {df_course.shape}")
    
    # Tạo set chứa firstnames từ course hiện tại
    course_usernames = set(df_course['firstname'].str.lower())
    print(f"Số lượng firstname trong {course}: {len(course_usernames)}")
    
    # Update level trong standard_df cho những firstname có trong course hiện tại
    level_name = level_mapping[course]
    mask = standard_df['firstname'].str.lower().isin(course_usernames)
    
    # Đếm số records sẽ được update
    update_count = mask.sum()
    print(f"Số records trong standard-data sẽ được gán level '{level_name}': {update_count}")
    
    # Update level
    standard_df.loc[mask, 'level'] = level_name
    
    # Hiển thị một vài ví dụ về records được update
    if update_count > 0:
        updated_records = standard_df[mask].head(3)
        print(f"Ví dụ records được gán level '{level_name}':")
        for idx, row in updated_records.iterrows():
            print(f"  - {row['username']} ({row['firstname']}) -> {row['level']}")

print(f"\n=== TỔNG KẾT ===")
print(f"Tổng số records trong standard-data: {len(standard_df)}")
print(f"Phân bố level sau khi update:")
level_counts = standard_df['level'].value_counts(dropna=False)
print(level_counts)

# Hiển thị sample kết quả cuối cùng
print(f"\nSample kết quả cuối cùng:")
print(standard_df[['username', 'firstname', 'lastname', 'level']].head(10))

Standard data shape: (4519, 4)
Standard data columns: ['username', 'firstname', 'lastname', 'email']

=== PROCESSING ETOP-BEGINNER ===
Course etop-beginner data shape after filtering: (2345, 3)
Số lượng firstname trong etop-beginner: 2345
Số records trong standard-data sẽ được gán level 'Beginner': 2367
Ví dụ records được gán level 'Beginner':
  - thuyduonghoi112 (AG16P0772) -> Beginner
  - duongthinhuy3006 (AG16P0774) -> Beginner
  - ngocthu25033012 (AG16P0775) -> Beginner

=== PROCESSING ETOP-INTERMEDIATE ===
Course etop-intermediate data shape after filtering: (1116, 3)
Số lượng firstname trong etop-intermediate: 1115
Số records trong standard-data sẽ được gán level 'Intermediate': 1147
Ví dụ records được gán level 'Intermediate':
  - kimdungphan555 (AG15P0730) -> Intermediate
  - auduongngoctam.0903 (AG15P0733) -> Intermediate
  - ngocthu25033012 (AG16P0775) -> Intermediate

=== PROCESSING ETOP-ADVANCED ===
Course etop-advanced data shape after filtering: (659, 3)
Số lượng firstnam

In [9]:
# Hiển thị kết quả dưới dạng HTML table với level information
print("=== SAMPLE OF UPDATED STANDARD DATA ===")
HTML(standard_df[['username', 'firstname', 'lastname', 'level']].head(15).to_html())

=== SAMPLE OF UPDATED STANDARD DATA ===


,username,firstname,lastname,level
0,phanthithukieu.22091994,AD22P0002,Phan Thị Kiều Hạnh,None
1,thichgame07,AD22P0003,Giáp Bảo Thiện,None
2,nghantran80,AD22P0004,Trần Ngọc Hân,None
3,ngthulptc,AD22P0005,Huỳnh Thị Ngọc Thư,None
4,thanhtruongtc00,AD22P0006,Trần Thanh Trường,None
5,huynhminhquanfpt188,AD22P0008,Huỳnh Minh Quân,None
6,nguyenhuynhkimthi1990,AD22P0009,Nguyễn Huỳnh Kim Thi,None
7,nphat1039,AD22P0010,Nguyễn Thành Phát,None
8,nguyentoan30112019,AD22P0011,Nguyễn Bảo Toàn,None
9,kimanh290719,AG15P0711,Nguyễn Thị Kim Anh,None


In [10]:
# Thống kê chi tiết về kết quả final
print("=== THỐNG KÊ TỔNG HỢP ===")
print(f"Tổng số records trong standard-data: {len(standard_df)}")

# Phân bố level
print(f"\n=== PHÂN BỐ LEVEL ===")
level_distribution = standard_df['level'].value_counts(dropna=False)
print(level_distribution)

# Tính phần trăm
total_records = len(standard_df)
for level, count in level_distribution.items():
    percentage = (count / total_records) * 100
    if pandas.isna(level):
        print(f"No Level assigned: {count} records ({percentage:.2f}%)")
    else:
        print(f"{level}: {count} records ({percentage:.2f}%)")

# Hiển thị ví dụ cho từng level
print(f"\n=== VÍ DỤ CHO TỪNG LEVEL ===")

for level in ['Beginner', 'Intermediate', 'Advanced']:
    level_records = standard_df[standard_df['level'] == level]
    if len(level_records) > 0:
        print(f"\n{level} Level (Total: {len(level_records)} records):")
        sample_records = level_records.head(3)
        for idx, row in sample_records.iterrows():
            print(f"  - {row['username']} | {row['firstname']} | {row['lastname']}")
    else:
        print(f"\n{level} Level: No records found")

# Records without level
no_level_records = standard_df[standard_df['level'].isna()]
if len(no_level_records) > 0:
    print(f"\nRecords without level (Total: {len(no_level_records)} records):")
    sample_no_level = no_level_records.head(3)
    for idx, row in sample_no_level.iterrows():
        print(f"  - {row['username']} | {row['firstname']} | {row['lastname']}")

print(f"\n=== SUMMARY ===")
print(f"✓ Successfully processed {len(data_course)} course levels")
print(f"✓ Total records with assigned levels: {standard_df['level'].notna().sum()}")
print(f"✓ Coverage rate: {(standard_df['level'].notna().sum() / len(standard_df) * 100):.2f}%")

=== THỐNG KÊ TỔNG HỢP ===
Tổng số records trong standard-data: 4519

=== PHÂN BỐ LEVEL ===
level
Beginner        1741
None            1305
Intermediate     805
Advanced         668
Name: count, dtype: int64
Beginner: 1741 records (38.53%)
No Level assigned: 1305 records (28.88%)
Intermediate: 805 records (17.81%)
Advanced: 668 records (14.78%)

=== VÍ DỤ CHO TỪNG LEVEL ===

Beginner Level (Total: 1741 records):
  - thuyduonghoi112 | AG16P0772 | Đỗ Thị Thùy Dương
  - duongthinhuy3006 | AG16P0774 | Dương Thị Như Ý
  - ngtram0334 | AG16P0779 | Nguyễn Thị Ngọc Trâm

Intermediate Level (Total: 805 records):
  - kimdungphan555 | AG15P0730 | Phan Nguyễn Kim Dung
  - auduongngoctam.0903 | AG15P0733 | Huỳnh Thị Kiều My
  - ngocthu25033012 | AG16P0775 | Lê Thị Ngọc Thư

Advanced Level (Total: 668 records):
  - nguyenlien170824 | AG18P0935 | Nguyễn Thị Thúy Liên
  - huynhthuytrang07012004 | AG18P0964 | Huỳnh Thị Thùy Trang
  - kimnganag091 | AG20U0004 | Lê Phan Minh Mỹ Kim Ngân

Records without l

In [11]:
# Save the updated standard data with level information
output_file = "./standard-data-with-levels.csv"
standard_df.to_csv(output_file, index=False)
print(f"✓ Updated standard data saved to: {output_file}")
print(f"✓ File contains {len(standard_df)} records with level information")

# Verify the saved file
verify_df = pandas.read_csv(output_file)
print(f"✓ Verification: Loaded {len(verify_df)} records from saved file")
print(f"✓ Columns in saved file: {verify_df.columns.tolist()}")
print(f"✓ Level distribution in saved file:")
print(verify_df['level'].value_counts(dropna=False))

✓ Updated standard data saved to: ./standard-data-with-levels.csv
✓ File contains 4519 records with level information
✓ Verification: Loaded 4519 records from saved file
✓ Columns in saved file: ['username', 'firstname', 'lastname', 'email', 'level']
✓ Level distribution in saved file:
level
Beginner        1741
NaN             1305
Intermediate     805
Advanced         668
Name: count, dtype: int64
